# Forecasting on Real-Time Hydrometric Data

Forecasting Using Regression Techniques

[Data Scource](https://wateroffice.ec.gc.ca/download/index_e.html?results_type=real_time)

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

set_config(display="diagram")
warnings.filterwarnings('ignore')

In [2]:
def df_info_func(df, vizualize=False, threshold=3, display_sample=True, side_by_side=True):
    def f1(series):
        try: return str(series.unique().tolist())
        except: return '--'

    print(f'Rows: {df.shape[0]} N Cols: {df.shape[1]}')

    if df.shape[0] > 0:
        df_info = pd.DataFrame(index=df.columns)
        df_info['data_types'] = df.dtypes.values
        df_info['n_missing'] = df.isna().sum().values    
        df_info['missing_pct'] = round((df_info['n_missing'] / len(df))*100,2)

        df_ = df.astype(str).copy()

        df_info['n_unique'] = df_.apply(lambda x: x.nunique(),axis=0).values
        df_info['uniq_vals'] = df_.apply(lambda x: np.where(x.nunique() <= threshold, f1(x), '--')).values

        for c in df.columns:            
            if df[c].dtype == 'datetime64[ns]':
                dt_str = "From: " + df[c].min().strftime('%d-%b-%Y')
                dt_str += " Till: " + df[c].max().strftime('%d-%b-%Y')
                df_info.loc[c,'uniq_vals'] = dt_str                
            elif df[c].dtype == 'float64' or  df[c].dtype == 'int64':
                real_num_series = df[c].dropna()
                if len(real_num_series) > 0:
                    dt_str = "Min: " + str(int(real_num_series.min()))
                    dt_str += " Max: " + str(int(real_num_series.max()))
                    dt_str += " Mean: " + str(int(real_num_series.mean()))
                    dt_str += " Med: " + str(int(real_num_series.median()))
                    dt_str += " Std: " + str(int(real_num_series.std()))
                else:
                    dt_str = ""
                df_info.loc[c,'uniq_vals'] = dt_str
            else:
                pass
                
        df_info.reset_index(inplace=True)
        df_info.rename(columns={'index':'cols'},inplace=True)
        if vizualize:
            # import seaborn as sns
            # sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')
            # or an another package to display missing values graph
            import missingno as msno 
            msno.matrix(df)

        if side_by_side:
            from IPython.display import display_html
            space = "\xa0" * 10
            info_styler = df_info.style.set_table_attributes("style='display:inline'").set_caption('DF Info')
            df_styler = df.head().style.set_table_attributes("style='display:inline'").set_caption('DF')
            display_html(info_styler._repr_html_() + space + df_styler._repr_html_(),raw=True)            
        else:
            from IPython.core.display import HTML
            display(HTML(df_info.to_html()))
            if display_sample: display(HTML(df.head().to_html()))
            
    else:
        print("Empty DataFrame")
        
def convert_ts_df(ts_df, shifts = 10):
    ts_df['year'] = ts_df.index.year
    ts_df['month'] = ts_df.index.month
    ts_df['day'] = ts_df.index.day
    ts_df['hour'] = ts_df.index.hour
    ts_df['minute'] = ts_df.index.minute
    ts_df['week_num'] = ts_df.index.week
    ts_df['week_day'] = ts_df.index.weekday
    
    diff_days = (ts_df.index - ts_df.reset_index().time.shift(1))
    diff_mins = diff_days.dt.total_seconds() // 60
    ts_df['prev_obs_min'] = diff_mins.tolist()  
    
    for i in range(1,shifts+1):
        ts_df['prev_val_'+str(i)] = ts_df['value'].shift(i)
    
    return ts_df
        
mape = lambda tr, pr: np.mean(np.abs((tr - pr) / tr)) * 100

# Data Import

In [3]:
df_org = pd.read_excel("datasets/WaterLevelData_Canada.xlsx")

# Data Cleaning

In [4]:
df = df_org.copy()
df.drop(columns=['Parameter '], inplace=True)
df.rename(columns={'Date (MST)':'time','Value (m)':'value'}, inplace=True)
df['time'] = pd.to_datetime(df['time'])
df.sort_values(by='time', inplace=True, ignore_index=True)
df_info_func(df)

Rows: 149688 N Cols: 2


DF Info cols data_types n_missing missing_pct n_unique uniq_vals 
 
 0 
 time 
 datetime64[ns] 
 0 
 0.000000 
 149688 
 From: 08-Jan-2020 Till: 01-Dec-2022 
 
 
 1 
 value 
 float64 
 0 
 0.000000 
 434 
 Min: 0 Max: 1 Mean: 0 Med: 0 Std: 0 
 
            DF time value 
 
 0 
 2020-01-08 00:00:00 
 0.984000 
 
 
 1 
 2020-01-08 00:05:00 
 0.984000 
 
 
 2 
 2020-01-08 00:10:00 
 0.984000 
 
 
 3 
 2020-01-08 00:15:00 
 0.985000 
 
 
 4 
 2020-01-08 00:20:00 
 0.985000

In [5]:
df.tail()

,time,value
149683,2022-12-01 23:35:00,1.127
149684,2022-12-01 23:40:00,1.126
149685,2022-12-01 23:45:00,1.126
149686,2022-12-01 23:50:00,1.122
149687,2022-12-01 23:55:00,1.120


some data is in future dates (Dec 22), might be a data quality problem.
so, from the below EDA we can remove these data points.

In [ ]:
# See if we can infer time
# Returns None as there is no temporal frequency
pd.infer_freq(df.time)

In [ ]:
diff_days = (df.time - df.time.shift(1))
diff_mins = diff_days.dt.total_seconds() // 60
diff_mins.value_counts(normalize=True).head()

As almost 98% of data is with a 5 minute frequency we can generate a data range with this freq

In [ ]:
df.set_index(keys='time', drop=True, inplace=True)
df_info_func(df)

In [ ]:
df_till_2021 = df.loc[:"2021"].copy()
df.loc["2021-12-21 15:10:00":].head()
df.loc["2021-12-21 15:10:00":].tail()
df_till_2021.head()

In [ ]:
# so there are no any timestamps with 2 values
# each 5 min had only one value and the rest are missing data where they are zeros
# Almost 29% of the data is missing from this
df_till_2021.resample('5min').count()['value'].value_counts(normalize=True)

In [ ]:
df_2021 = df_till_2021.resample('5min',).mean()
df_2021.head()

# Visual Inspection

In [ ]:
plt.figure(figsize=(15,3))
plt.plot(df_2021)
plt.show();

plt.figure(figsize=(15,3))
plt.plot(df_2021.loc["2020"])
plt.plot(df_2021.loc["2021"])
plt.show();

In [ ]:
years = df_2021.index.year.unique().tolist()

fig, axes = plt.subplots(len(years), 1)
for idx, yr in enumerate(years):    
    axes[idx].plot(df_2021.loc[str(yr)])
    axes[idx].set_title(f"Year-{yr}")    
plt.show();

In [ ]:
years = df_2021.index.year.unique().tolist()
months = range(1,13)

fig, axes = plt.subplots(len(years), len(months), figsize=(15, 5))

for idx_yr, yr in enumerate(years):
    for idx_mn, mon in enumerate(months):
        axes[idx_yr, idx_mn].plot(df_2021.loc[str(yr)+"-"+str(mon)].asfreq("5min").fillna(0))
        axes[idx_yr, idx_mn].set_title(f"{yr}-{mon}")
        # axes[idx_yr, idx_mn].axis('off')
        axes[idx_yr, idx_mn].get_xaxis().set_visible(False)

plt.show();

In [ ]:
df_info_func(df_2021)

In [ ]:
df_full = df_2021.dropna().copy()
df_info_func(df_full)

# Data Prep

In [ ]:
alfa = convert_ts_df(df_full)
df_info_func(alfa)

In [ ]:
# Number of 5min intervals in 7 days
max_clip_val = (60/5) * 24 * 7
max_clip_val

In [ ]:
alfa['prev_obs_min'] = alfa['prev_obs_min'].clip(0,max_clip_val)

In [ ]:
alfa.dropna(inplace=True)
X = alfa.drop(columns=['value'])
y = alfa['value']
X.shape, y.shape

# Model Building

## Data Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Building Data Preprocessing Pipeline and model Training

In [ ]:
preprocessor = ColumnTransformer(transformers=[("num", MinMaxScaler(), X.columns),])
model_pipe = Pipeline(steps=[("preprocessor", preprocessor), ("reg", LinearRegression())])
param_grid = {"reg__fit_intercept": [True, False],}
model_grid = RandomizedSearchCV(model_pipe, param_grid, cv=10)
model_grid.fit(X_train,y_train)
model_grid

In [ ]:
model_grid.best_params_

## Evaluation

In [ ]:
print(f"Internal CV score: {model_grid.best_score_:.3f}")

In [ ]:
cv_results = pd.DataFrame(model_grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[["rank_test_score","mean_test_score","std_test_score","params"]]

In [ ]:
print("Regression Model","\n\n")
y_pred = model_grid.predict(X_test)
errors = y_test - y_pred

print("mean_absolute_error: ",mean_absolute_error(y_test, y_pred))
print("mean_squared_error: ",mean_squared_error(y_test, y_pred))
print("mape: ",mape(y_test, y_pred))
print("r2_score: ",r2_score(y_test, y_pred))

### Compare our model with a base line

In [ ]:
print("Baseline Model","\n\n")
y_baseline = y.mean()
y_preds_baseline = [y_baseline]* len(y_test)

print("BaseLine mean_absolute_error: ",mean_absolute_error(y_test, y_preds_baseline))
print("BaseLine mean_squared_error: ",mean_squared_error(y_test, y_preds_baseline))
print("BaseLine mape: ",mape(y_test, y_preds_baseline))
print("BaseLine r2_score: ",r2_score(y_test, y_preds_baseline))

### Viz of results

In [ ]:
plt.scatter(y_test, y_pred)
plt.show();
plt.boxplot(errors, vert=False)
plt.show();

## Feature Importances (based on Linear Regression model)

In [ ]:
feat_imp_scores = model_grid.best_estimator_.named_steps['reg'].coef_
plt.barh(X.columns, feat_imp_scores)

In [ ]:
%matplotlib inline
print(X_test.prev_obs_min.unique())
plt.scatter(X_test.prev_obs_min, errors)
plt.xlim(0,75)
plt.show();

# Future Predictions

Do an one step ahead forecasting. for next 7 days. 

In [ ]:
start_time = alfa.index.max() + pd.Timedelta("5min")
start_time

In [ ]:
alfa.head(1)

In [ ]:
pred_range = pd.date_range(start_time , freq='5min', periods=max_clip_val)
beta = pd.DataFrame(index=pred_range)
beta['year'] = beta.index.year
beta['month'] = beta.index.month
beta['day'] = beta.index.day
beta['hour'] = beta.index.hour
beta['minute'] = beta.index.minute
beta['week_num'] = beta.index.week
beta['week_day'] = beta.index.weekday
diff_days = beta.index - alfa.index.max()
diff_mins = diff_days.total_seconds() // 60
beta['prev_obs_min'] = diff_mins

beta['prev_obs_min'] = beta['prev_obs_min'].clip(0,max_clip_val)

prev_vals = alfa.loc[alfa.index.max()]
for i in range(1,11):
    beta['prev_val_'+str(i)] = prev_vals['prev_val_'+str(i)]    
beta.head()

In [ ]:
future_pred_vals = model_grid.predict(beta)
beta['future_pred_vals'] = future_pred_vals
beta.head(2)

In [ ]:
plt.plot(beta.index,future_pred_vals);

# Next Steps:
* Design Schema for DB
* Push Prediction to DB
* Build API
* Deploy in instance
* Testing

### Secondary Items:
* convert to Py scripts
* object oriented / reusable scripts
* Integrate multiple sensors logic
* Re-training schedule